In [1]:
import yaml
from matplotlib import pyplot as plt
%matplotlib inline

from utils.load_datasets import load_dataset
from models import vgg
from models.cvae import CVAE
from models.vgg_vmmix import BiternionVGGMixture

Using TensorFlow backend.


In [2]:
dataset_name = "TownCentre" #"IDIAP"
dataset_path = 'data/TownCentre.pkl.gz' #'data//IDIAP.pkl'
net_output = 'pan'
model_type = 'cvae'

In [3]:
(xtr, ytr_bit, ytr_deg), (xval, yval_bit, yval_deg), (xte, yte_bit, yte_deg) = load_dataset(dataset_name,
                                                                                            dataset_path,
                                                                                            net_output)

image_height, image_width, n_channels = xtr.shape[1], xtr.shape[2], xtr.shape[3]

In [4]:
net_config_path = '/Users/sergey/BiternionNet/pretrained_models/TownCentre/cvae/model.trial_30.params.yml'
net_weights_path = '/Users/sergey/BiternionNet/pretrained_models/TownCentre/cvae/model.trial_30.best.weights.hdf5'

In [22]:
#net_config_path = '/Users/sergey/BiternionNet/pretrained_models/IDIAP/vm_mixture/model.trial_0.params.yml'
#net_weights_path = '/Users/sergey/BiternionNet/pretrained_models/IDIAP/vm_mixture/model.trial_0.best.weights.hdf5'

In [5]:
with open(net_config_path, 'r') as f:
    net_config = yaml.load(f)

if model_type == 'vm_mixture':
    model = BiternionVGGMixture(image_height=image_height,
                            image_width=image_width,
                            n_channels=3,
                            **net_config)
    model.load_weights(net_weights_path)
    
elif model_type == 'cvae':
    model = CVAE(image_height=image_height,
                 image_width=image_width,
                 n_channels=n_channels,
                 **net_config)
    model.load_weights(net_weights_path)

In [7]:
model.evaluate(xte[0:1000], yte_deg[0:1000], 'test')

MAAD error (test) : 21.711964 pm 1.031099SEM
ELBO (test) : -0.544346 pm 0.058636SEM
Approx Log-Likelihood, importance sampling (test) : -0.543643 pm 0.058561SEM
KL-div (test) : 0.000025 pm0.000002SEM


{'elbo': -0.54434585906501576,
 'elbo_sem': array([ 0.05863604]),
 'importance_log_likelihood': -0.54364339532317485,
 'importance_log_likelihood_sem': 0.058561346512496101,
 'kl_div': 2.5416994473700863e-05,
 'kl_div_sem': array([  2.24512186e-06]),
 'maad_loss': 21.711963826291122,
 'maad_loss_sem': 1.0310986475480064}

In [17]:
cvae_preds = model.get_multiple_predictions(xte[0:10], yte_bit[0:10], n_samples=50)

In [21]:
cvae_preds['mu_rad_dec'][3]

array([ 2.50098155,  2.50141369,  2.50240127,  2.49910225,  2.50134658,
        2.50081634,  2.5015735 ,  2.50256231,  2.49989361,  2.50244983,
        2.50009991,  2.50162242,  2.50093397,  2.50260379,  2.50561201,
        2.50208966,  2.50122677,  2.50409839,  2.49476727,  2.50105916,
        2.5027754 ,  2.50433038,  2.50240625,  2.50286551,  2.50076805,
        2.50216884,  2.50249854,  2.50146726,  2.50533915,  2.50049632,
        2.50369443,  2.50393745,  2.50215236,  2.49965775,  2.49991205,
        2.50242193,  2.49698801,  2.49549138,  2.5033691 ,  2.50020553,
        2.5019556 ,  2.50521052,  2.50195508,  2.5018183 ,  2.50372495,
        2.50217749,  2.50225945,  2.50501805,  2.4982076 ,  2.49893308])

In [ ]:
model.evaluate()